In [19]:
import numpy as np
from sklearn.svm import SVC

In [37]:
def get_data(n_class, perc):
    file_name = f"n_class_{n_class}_maj_perc_0_{perc}.txt"
    train_dir = f"../expert_load_train_ImageNetData_enc"
    test_dir = f"../expert_load_test_ImageNetData_enc"
    train_x, train_y = [], []

    with open(f"{train_dir}/{file_name}", "r") as f:
        for line in f:
            x_y = line.strip().split(",")
            if x_y[0] == 'l5e1':
                continue
            train_x.append(x_y[:16])
            train_y.append(x_y[16])

    test_x, test_y = [], []

    with open(f"{test_dir}/{file_name}", "r") as f:
        for line in f:
            x_y = line.strip().split(",")
            if x_y[0] == 'l5e1':
                continue
            test_x.append(x_y[:16])
            test_y.append(x_y[16])
    return train_x, train_y, test_x, test_y

In [46]:
for n_class in [2, 4, 6, 8, 16, 32, 64]:
    for perc in [25, 375, 5, 625, 875]:
        train_x, train_y, test_x, test_y = get_data(64,25)

        clf = SVC(kernel='linear')
        clf = clf.fit(train_x, train_y)

        pred = clf.score(test_x)
        print(f"n_class: {n_class}, perc: {perc}, accuracy: {pred}")

['505' '909' '724' '139' '207' '444' '709' '143' '139' '444' '690' '139'
 '945' '237' '297' '991' '453' '597' '444' '139' '14' '709' '148' '194'
 '709' '303' '957' '709' '540' '444' '887' '235' '307' '540' '846' '709'
 '709' '783' '148' '143' '178' '846' '351' '222' '458' '235' '564' '583'
 '846' '402' '28' '583' '690' '846' '690' '303' '887' '505' '207' '76'
 '139' '505' '645' '390' '721' '444' '410' '453' '583' '349' '61' '721'
 '709' '333' '237' '237' '485' '444' '178' '597' '709' '139' '723' '540'
 '349' '361' '194' '580' '139' '351' '945' '14' '237' '390' '235' '453'
 '390' '947' '180' '297' '222' '709' '207' '178' '724' '180' '709' '723'
 '485' '723' '180' '583' '178' '222' '178' '444' '351' '235' '143' '916'
 '237' '583' '909' '472' '453' '846' '143' '410' '178' '947' '583' '237'
 '76' '709' '207' '485' '741' '540' '390' '297' '709' '237' '351' '721'
 '991' '143' '351' '178' '336' '235' '690' '762' '297' '402' '690' '444'
 '351' '444' '180' '336' '143' '709' '947' '458' '976' '3

In [47]:
print(np.mean(pred == test_y))    

0.3411458333333333


In [51]:
labels = list(set(test_y))
labels.sort()
random_guess_y = np.random.choice(labels, len(test_y))
print(np.mean(random_guess_y == test_y))    

0.013020833333333334
